In [193]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [194]:
token = pd.read_parquet('token.parquet')

In [195]:
library = pd.read_parquet('library.parquet')

## Cleanup

In [196]:
token

pos  token_str   term_str
speech_id speaker  para_id sent_id token_id                           
0         CLINTON  0       0       0          NN        Wow        wow
                           1       0          NN      Thank      thank
                                   1         PRP        you        you
                           2       0         NNP      Thank      thank
                                   1         PRP        you        you
...                                          ...        ...        ...
162       CLINTON  273     1       3         PRP        you        you
          MATTHEWS 274     0       0         NNP      Thank      thank
                                   1         PRP       you,        you
                                   2         NNP      Madam      madam
                                   3         NNP  Secretary  secretary

[555033 rows x 3 columns]

In [197]:
#Split into Trump and Clinton speeches
clinton = token.query('speaker=="CLINTON"')
trump = token.query('speaker=="TRUMP"')

In [198]:
#Remove paragraph and sentence identifiers since we won't be using them
clinton=clinton.droplevel([3,4])
trump=trump.droplevel([3,4])

In [199]:
clinton

pos token_str term_str
speech_id speaker para_id                        
0         CLINTON 0         NN       Wow      wow
                  0         NN     Thank    thank
                  0        PRP       you      you
                  0        NNP     Thank    thank
                  0        PRP       you      you
...                        ...       ...      ...
162       CLINTON 273      NNP     Chris    chris
                  273       NN     Great    great
                  273       TO        to       to
                  273       VB       see      see
                  273      PRP       you      you

[148800 rows x 3 columns]

In [200]:
#Combine terms to create a single row for each speech, add the speaker
clinton = clinton.groupby('speech_id')['term_str'].apply(' '.join).reset_index()
trump = trump.groupby('speech_id')['term_str'].apply(' '.join).reset_index()
clinton['speaker']='clinton'
trump['speaker']='trump'

In [201]:
clinton

,speech_id,term_str,speaker
0,0,wow thank you thank you all thank you i am rea...,clinton
1,1,thank you thank you all very much thank you th...,clinton
2,2,thanks george look we are definitely in confli...,clinton
3,3,right right well yes right no because he got a...,clinton
4,4,thank you thank you very much applause thank y...,clinton
5,5,thank you mr chairman ranking member cummings ...,clinton
6,6,he thought i did a really good job and i think...,clinton
7,7,he thought i did a really good job and i think...,clinton
8,8,i think where we are today is due in large mea...,clinton
9,9,thank you very much as far as i know thats tru...,clinton


In [202]:
#Create combined dataframes
speeches=clinton.append(trump)

In [203]:
speeches=speeches.set_index('speech_id').sort_index()

## Compare to Scikit-Learn's MultinomialNB

In [204]:
#Split into training and test data
speech_train, speech_test, speaker_train, speaker_test = train_test_split(speeches.term_str, speeches.speaker, test_size=0.2, random_state=42)

In [208]:
model = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB())])

In [209]:
model.fit(speech_train, speaker_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [210]:
predicted = text_clf.predict(speech_test)
np.mean(predicted == speaker_test)

0.9047619047619048